In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql.types import StringType, MapType
mapCol = MapType(StringType(),StringType(),False)

In [3]:
"""
1. 첫 번째 매개변수는 keyType맵에서 키 유형을 지정하는 데 사용됩니다.
2. 두 번째 매개변수는 valueType맵에서 값의 유형을 지정하는 데 사용됩니다.
3. 세 번째 매개변수는 valueContainsNull두 번째 매개변수의 값이 값을 수락할 수 있는지 
지정하는 데 사용되는 선택적 부울 유형입니다 Null/None.
4. 지도의 키는 None/Null값을 허용하지 않습니다.
5. PySpark는 작업할 여러 SQL 함수를 제공합니다 MapType.
"""

'\n1. 첫 번째 매개변수는 keyType맵에서 키 유형을 지정하는 데 사용됩니다.\n2. 두 번째 매개변수는 valueType맵에서 값의 유형을 지정하는 데 사용됩니다.\n3. 세 번째 매개변수는 valueContainsNull두 번째 매개변수의 값이 값을 수락할 수 있는지 \n지정하는 데 사용되는 선택적 부울 유형입니다 Null/None.\n4. 지도의 키는 None/Null값을 허용하지 않습니다.\n5. PySpark는 작업할 여러 SQL 함수를 제공합니다 MapType.\n'

In [8]:
from pyspark.sql.types import StructField, StructType, StringType, MapType
schema = StructType([
    StructField('name', StringType(), True),
    StructField('properties', MapType(StringType(),StringType()),True)
])

In [9]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()
dataDictionary = [
        ('James',{'hair':'black','eye':'brown'}),
        ('Michael',{'hair':'brown','eye':None}),
        ('Robert',{'hair':'red','eye':'black'}),
        ('Washington',{'hair':'grey','eye':'grey'}),
        ('Jefferson',{'hair':'brown','eye':''})
        ]
df = spark.createDataFrame(data=dataDictionary, schema = schema)
df.printSchema()
df.show(truncate=False)

root
 |-- name: string (nullable = true)
 |-- properties: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)

+----------+-----------------------------+
|name      |properties                   |
+----------+-----------------------------+
|James     |{eye -> brown, hair -> black}|
|Michael   |{eye -> null, hair -> brown} |
|Robert    |{eye -> black, hair -> red}  |
|Washington|{eye -> grey, hair -> grey}  |
|Jefferson |{eye -> , hair -> brown}     |
+----------+-----------------------------+



In [10]:
df3=df.rdd.map(lambda x: \
    (x.name,x.properties["hair"],x.properties["eye"])) \
    .toDF(["name","hair","eye"])
df3.printSchema()
df3.show()

root
 |-- name: string (nullable = true)
 |-- hair: string (nullable = true)
 |-- eye: string (nullable = true)

+----------+-----+-----+
|      name| hair|  eye|
+----------+-----+-----+
|     James|black|brown|
|   Michael|brown| null|
|    Robert|  red|black|
|Washington| grey| grey|
| Jefferson|brown|     |
+----------+-----+-----+



In [11]:
# Using getItem
df.withColumn("hair",df.properties.getItem("hair")) \
  .withColumn("eye",df.properties.getItem("eye")) \
  .drop("properties") \
  .show()

# Using properties
df.withColumn("hair",df.properties["hair"]) \
  .withColumn("eye",df.properties["eye"]) \
  .drop("properties") \
  .show()

+----------+-----+-----+
|      name| hair|  eye|
+----------+-----+-----+
|     James|black|brown|
|   Michael|brown| null|
|    Robert|  red|black|
|Washington| grey| grey|
| Jefferson|brown|     |
+----------+-----+-----+

+----------+-----+-----+
|      name| hair|  eye|
+----------+-----+-----+
|     James|black|brown|
|   Michael|brown| null|
|    Robert|  red|black|
|Washington| grey| grey|
| Jefferson|brown|     |
+----------+-----+-----+



In [12]:
# Explode

from pyspark.sql.functions import explode
df.select(df.name,explode(df.properties)).show()

+----------+----+-----+
|      name| key|value|
+----------+----+-----+
|     James| eye|brown|
|     James|hair|black|
|   Michael| eye| null|
|   Michael|hair|brown|
|    Robert| eye|black|
|    Robert|hair|  red|
|Washington| eye| grey|
|Washington|hair| grey|
| Jefferson| eye|     |
| Jefferson|hair|brown|
+----------+----+-----+



In [13]:
# map_keys - take all map key
# 경고 : 매우 느리게 실행

from pyspark.sql.functions import map_keys
df.select(df.name,map_keys(df.properties)).show()

+----------+--------------------+
|      name|map_keys(properties)|
+----------+--------------------+
|     James|         [eye, hair]|
|   Michael|         [eye, hair]|
|    Robert|         [eye, hair]|
|Washington|         [eye, hair]|
| Jefferson|         [eye, hair]|
+----------+--------------------+



In [ ]:
# map_values() - take all map value

from pyspark.sql.functions import map_values
df.select(df.name,map_values(df.properties)).show()